# Машинное обучение

## Факультет математики НИУ ВШЭ

### 2018-2019 учебный год

Лектор: Илья Щуров

Семинаристы: Евгения Ческидова, Евгений Ковалев

Ассистенты: Константин Ваниев, Софья Дымченко

# Семинар 13

На этом семинаре мы:

- решим задачу классификации с помощью рекуррентной нейронной сети

Используем датасет с именами и будем определять, из какого языка произошло данное имя. Для этого построим рекуррентную сеть, которая работает с именами на уровне символов. А именно, на вход сети подается набор символов (имя), и на каждом шаге на выходе из сети оказываются предсказание и скрытое состояние, которое используется на следующем шаге. Предсказание на последнем шаге будет браться за итоговое, то есть мы получим прогноз сети, к какому классу принадлежит данное имя.

Ссылка для скачивания: https://download.pytorch.org/tutorial/data.zip

### Обработка данных

Для начала приведем текстовые файлы с именами к удобному формату, так, что на выходе получится словарь язык-список имен. В код можно особо не вникать.

In [ ]:
from io import open
import numpy as np
import os

import unicodedata
import string

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

# Turn a Unicode string to plain ASCII, thanks to http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

print(unicodeToAscii('Ślusàrski'))

# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

names_path = 'data/names/'

print(os.listdir(names_path))


# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

for filename in os.listdir(names_path):
    category = filename.split('.')[0]
    all_categories.append(category)
    lines = readLines(names_path + filename)
    category_lines[category] = lines

n_categories = len(all_categories)

In [ ]:
print(np.random.choice(category_lines['Korean'], size=7, replace=False))
print(np.random.choice(category_lines['Russian'], size=7, replace=False))

In [ ]:
len(all_letters)

Конвертируем имена в тензоры для того, чтобы их можно было подать на вход сети. Для этого векторизуем каждую букву бинарным вектором из нулей и единиц с 1 на позиции, соответствующей индексу буквы в алфавите. Например, `"c" -> (0 0 1 0 0 ...)`. Таким образом, имя превратится в тензор размера `количество символов x величина батча x количество букв в алфавите`. В данном случае будем использовать 1 батч, букв в алфавите всего 57, поэтому итоговая размерность будет `количество символов x 1 x 57`.

In [ ]:
import torch

# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    # (づ｡◕‿‿◕｡)づ

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    # (づ｡◕‿‿◕｡)づ

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    # (づ｡◕‿‿◕｡)づ

print(letterToTensor('J'))

print(lineToTensor('Jones').size())

### Построение модели

![image](https://i.imgur.com/Z2xbySO.png)

Реализуем простейшую однослойную RNN.

In [ ]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.activation = nn.ReLU()
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), dim=1)
        # (づ｡◕‿‿◕｡)づ
        return output, hidden

    def initHidden(self):
        # (づ｡◕‿‿◕｡)づ

n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

Проверим, что все корректно работает: выходы классификатора должны быть лог-вероятностями (чем больше, тем вероятнее категория).

In [ ]:
input = letterToTensor('A')
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input, hidden)

In [ ]:
input = lineToTensor('Albert')
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input[0], hidden)
print(output)
print(torch.exp(output).sum())

### Обучение

Чтобы интерпретировать выход модели, напишем функцию, переводящую лог-вероятности в категорию.

In [ ]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return all_categories[category_i], category_i

print(categoryFromOutput(output))

Функция для получения случайного объекта из обучающей выборки.

In [ ]:
import random

def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor

for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category, '/ line =', line)

Теперь перейдем непосредственно к обучению. Для этого напишем функцию, в ходе которой:

- на вход поступают объект (имя) и класс (язык)
- инициализируются скрытые состояния (нулями)
- forward-pass: считывается каждый символ и скрытое состояние для него сохраняется для следующего символа
- считается итоговое предсказание
- считается значение функции потерь (loss)
- backward-pass и обновление весов
- возвращаются loss и качество предсказания (accuracy)

In [ ]:
def train(category, category_tensor, line_tensor, optimizer):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    loss = F.nll_loss(output, category_tensor)
    loss.backward()
    optimizer.step()

    acc = (categoryFromOutput(output)[0] == category)

    return loss.item(), acc

Теперь обучим построенную модель на случайных объектах из обучающей выборки и построим графики значений функции потерь и качества. На каждом тысячном шагу будем ставить отметку, соответствующую среднему значению на предыдущих `1000` объектах.

In [ ]:
from tqdm import trange
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

n_iters = 50000
plot_every = 1000

current_loss = 0
all_losses = []
current_acc = 0
all_accs = []

n_hidden = 128

rnn = RNN(n_letters, n_hidden, n_categories)
opt = torch.optim.RMSprop(rnn.parameters(), lr=0.001)
for iter in trange(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    loss, acc = train(category, category_tensor, line_tensor, opt)
    current_loss += loss
    current_acc += acc

    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0
        all_accs.append(current_acc / plot_every)
        current_acc = 0

        
plt.figure()
plt.title("Loss")
plt.plot(range(plot_every, n_iters + 1, plot_every), all_losses)
plt.grid()
plt.show()

plt.figure()
plt.title("Accuracy")
plt.plot(range(plot_every, n_iters + 1, plot_every), all_accs)
plt.grid()
plt.show()

### Результат

Сделаем предсказание сети для `10000` случайных объектов. Для визуализации результата построим матрицу ошибок для языков.

In [ ]:
import matplotlib.ticker as ticker

# Keep track of correct guesses in a confusion matrix
confusion = torch.zeros(n_categories, n_categories)
n_confusion = 10000

# Just return an output given a line
def evaluate(line_tensor):
    hidden = rnn.initHidden()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    return output

# Go through a bunch of examples and record which are correctly guessed
for i in range(n_confusion):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output = evaluate(line_tensor)
    guess, guess_i = categoryFromOutput(output)
    category_i = all_categories.index(category)
    confusion[category_i][guess_i] += 1

# Normalize by dividing every row by its sum
for i in range(n_categories):
    confusion[i] = confusion[i] / confusion[i].sum()

# Set up plot
fig = plt.figure(figsize=(16,6))
ax = fig.add_subplot(111)
cax = ax.matshow(confusion.numpy())
fig.colorbar(cax)

# Set up axes
ax.set_xticklabels([''] + all_categories, rotation=90)
ax.set_yticklabels([''] + all_categories)

# Force label at every tick
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

# sphinx_gallery_thumbnail_number = 2
plt.show()

Время поэкспериментировать с предсказаниями!

In [ ]:
def predict(input_line, n_predictions=5):
    print('\n> %s' % input_line)
    with torch.no_grad():
        output = evaluate(lineToTensor(input_line))

        # Get top N categories
        topv, topi = output.topk(n_predictions, 1, True)
        predictions = []

        for i in range(n_predictions):
            value = topv[0][i].item()
            category_index = topi[0][i].item()
            print('(%.2f) %s' % (value, all_categories[category_index]))
            predictions.append([value, all_categories[category_index]])

predict('Hinton')

### To do

- измените параметр `learning_rate` внутри оптимизатора. Что происходит, когда он слишком большой? А слишком маленький?

- добавьте больше слоев!

- решите эту же задачу с помощью LSTM и GRU блоков (`nn.LSTM`, `nn.GRU`)

- **(*)** постройте более сложные модели на основе уже испробованных - например, двусторонние (bidirectional) LSTM и GRU